In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from json import loads


url = 'https://www.youtube.com/@PW-Foundation/videos'

# Get the html by get method
r = requests.get(url)
print(r)

# Create BeautifulSoup object
soup = BeautifulSoup(r.text, 'html.parser')
all_script_tags = soup.findAll('script')

<Response [200]>


In [2]:
def script_tag_to_json(tags: list) -> dict:
    for tag in reversed(tags):
        text: str = tag.text
        if 'ytInitialData = {"responseContext"' in text:
            return loads(text[20:-1])

    raise ValueError('Required script tag not found in the given tags.')


In [3]:
data = script_tag_to_json(all_script_tags)

#Return data from videos
def get_contents_dict(data):
    return data['contents']['twoColumnBrowseResultsRenderer']['tabs'][1]['tabRenderer']['content']['richGridRenderer']['contents']

In [ ]:
'contents'
    'twoColumnBrowseResultsRenderer'
        'tabs'
            [1]
                'tabRenderer'
                    'content'
                        'richGridRenderer'
                            'contents'

# Q1. Write a python program to extract the video URL of the first five videos.

In [10]:
def get_videoUrl(data:dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append('https://www.youtube.com/watch?v=' +
                      contents[i]['richItemRenderer']['content']['videoRenderer']['videoId'])

    return result

get_videoUrl(data)

['https://www.youtube.com/watch?v=aPrSAvwuhTk',
 'https://www.youtube.com/watch?v=tigMKfSBJ9E',
 'https://www.youtube.com/watch?v=2OrYDCb737E',
 'https://www.youtube.com/watch?v=U1FxvG4GNAY',
 'https://www.youtube.com/watch?v=t961YvjKH20']

# Q2. Write a python program to extract the URL of the video thumbnails of the first five videos.

In [11]:
def get_thumbnails(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['thumbnail']['thumbnails'][-1]['url'])

    return result

get_thumbnails(data)

['https://i.ytimg.com/vi/aPrSAvwuhTk/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAHjyITXGQem-Qr_DLUwa8_ARs1uw',
 'https://i.ytimg.com/vi/tigMKfSBJ9E/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLB_HwT5mDgp29rRJrLZPtEJvhys3g',
 'https://i.ytimg.com/vi/2OrYDCb737E/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLBOIrYBXFYvjYZBd-dtQzYUUvWlOQ',
 'https://i.ytimg.com/vi/U1FxvG4GNAY/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCp50JQzucjgWX6eHp1zj7dziat_g',
 'https://i.ytimg.com/vi/t961YvjKH20/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLB65fW1XKTh4z5bJEM5ndiGYWkWvA']


# Q3. Write a python program to extract the title of the first five videos.

In [12]:
def get_title(data: dict, n:int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['title']['runs'][-1]['text'])

    return result

get_title(data)

['How to change Mindset from class 10th to 11th ❓ ❓',
 'Bachoo आपके क्या Plan है इस Summer Vacation ❓❓',
 'The Ultimate Class 11th Survival Guide: Tips and Tricks for Success',
 'If not MBBS then What ❓|| Career Options for Biology Students',
 'Why Commerce After 10th, Explore the Best Career Options | Must Watch']

# Q4. Write a python program to extract the number of views of the first five videos.

In [13]:
def get_viwes(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(int(contents[i]['richItemRenderer']['content']['videoRenderer']['viewCountText']['simpleText']
                      [:-6].replace(',', '')))

    return result

get_viwes(data)

[6740, 30638, 127260, 33911, 32312]

# Q5. Write a python program to extract the time of posting of video for the first five videos.

In [14]:
def get_time_of_posting(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['publishedTimeText']['simpleText'])

    return result

get_time_of_posting(data)

['2 hours ago', '1 day ago', '3 days ago', '3 days ago', '5 days ago']

# Note: Save all the data scraped in the above questions in a CSV file.

In [15]:
def get_channel_video_details(data: dict, n: int):
    thumbnails = get_thumbnails(data, n)
    time_of_posting = get_time_of_posting(data, n)
    titles = get_title(data, n)
    video_urls = get_videoUrl(data, n)

    main_data = list(zip(video_urls, titles, thumbnails, time_of_posting))
    
    df = DataFrame.from_dict(main_data)
    df.rename(
        columns={
            0: 'video_urls',
            1: 'title',
            2: 'thumbnail_url',
            3: 'time_of_posting'
        }, inplace=True)

    return df

channel_data = get_channel_video_details(data, 30)
channel_data.to_csv('PW-Foundation.csv', index=False)
